In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import psycopg2
import glob

from datetime import date


In [2]:
# Function to get current working directory
def current_path():
    print("Current Directory:")
    print(os.getcwd())
    print()
    
# Function to get new directory 
def new_path():
    print("New Directory:")
    print(os.getcwd())
    print()
 
 
# Driver's code
# Printing CWD before
current_path()
 
# Changing the CWD
os.chdir('../../../Work Order - Data/WO_Folders/WO_2022-2023')
 
# Printing CWD after
new_path()

#Itererates and gets the latest file
cwd = os.getcwd()
files = [x for x in os.listdir(cwd) if x.endswith(".csv")]
newest_file = max(files, key = os.path.getctime)
print("Latest CSV Received: ", newest_file)

#Ensuring that the files is chosen:
os.path.exists(newest_file)

Current Directory:
C:\Users\E178162\Documents\Work Orders\DB\Jupyter Notebooks\Data Cleanse

New Directory:
C:\Users\E178162\Documents\Work Orders\Work Order - Data\WO_Folders\WO_2022-2023

Latest CSV Received:  Work_Order_(04_28_2023_03_54_18_PM).csv


True

In [3]:
dtype_dict = {'Work Order #': str}

df = pd.read_csv(newest_file, dtype=dtype_dict)

In [4]:
df['Date Open'] = pd.to_datetime(df['Created On']).dt.date
df['Date Closed'] = pd.to_datetime(df['Completed']).dt.date #Creates new column and sets data type from DateTime -> Date

df['Date Difference'] = date.today() - df['Date Open'] #Takes the difference between today's date from Date Open
df['Date Difference'] = df['Date Difference'].dt.days #Converts DateTime Type to Integer

df['Duration'] = df['Date Closed'] - df['Date Open'] #Takes the diffrence between Date Closed from Date Open
df['Duration'] = df['Duration'].dt.days #Converts DateTime Type to Integer
df['Duration'] = df['Duration'].abs() #Gets the Absolute Value of Column Values

df['Site_ID'] = df['Source Site'].str[-3:] #Takes the last 3 characters froms string in this case, we get extract the Site Code

df['School Year'] = '2022 - 2023' #Create new column of School Year

df['Duration'] = df['Duration'].fillna(df['Date Difference']) #Fill null values from Duration column with values from Date Difference

df['Row Count'] = 1 #Create new column of Row Count

df = df.drop(columns=['Area/Room #', 'Originator', 'Originator First Name', 'Work requested', 'Cause', 'Action Taken', 'Comments' ,'Longitude', 'Latitude', 'PM Title', 'Completed','Created On', 'Originated']) #Drops/Removes the columns from table

df = df[df['Site_ID'].str.contains("ZZ-|032|941|ium") == False] #Remove any string or cell that contains those characters

df = df.drop_duplicates(subset='Work Order #', keep='first') #Remove any duplicates

# Delete rows where work order is present but work category is null
df = df.dropna(subset=['Work Category'])

df = df.rename(columns={"Work Order #": "Work Order ID"})

df

columns = ['Work Order ID', 'WO Status', 'Priority', 'Work Category', 'Problem',
              'Source Site', 'Site_ID' , 'Source Location', 'Source User', 
              'Date Open', 'Date Closed', 'Date Difference', 'Duration',
              'School Year','Work Type', 'Labor Hrs', 'Part Cost', 'Labor Cost',
              'Total Hrs', 'Grand Total', 'Row Count'] 

df = df[columns]

df['Site_ID'] = df['Site_ID'].astype(int)

df["Source User"].fillna("Pending Assignment", inplace=True)

work_order_report = df

work_order_report.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45053 entries, 0 to 45075
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Work Order ID    45053 non-null  object 
 1   WO Status        45053 non-null  object 
 2   Priority         43169 non-null  object 
 3   Work Category    45053 non-null  object 
 4   Problem          45053 non-null  object 
 5   Source Site      45053 non-null  object 
 6   Site_ID          45053 non-null  int32  
 7   Source Location  43394 non-null  object 
 8   Source User      45053 non-null  object 
 9   Date Open        45053 non-null  object 
 10  Date Closed      39834 non-null  object 
 11  Date Difference  45053 non-null  int64  
 12  Duration         45053 non-null  float64
 13  School Year      45053 non-null  object 
 14  Work Type        23979 non-null  object 
 15  Labor Hrs        45053 non-null  float64
 16  Part Cost        45053 non-null  float64
 17  Labor Cost  

In [5]:
null_category_wos = df.loc[df['Work Category'].isnull(), 'Work Order ID']
print(null_category_wos)

Series([], Name: Work Order ID, dtype: object)


In [6]:
# Upload each dataframe into separate tables
dataframes = [work_order_report]
table_names = ["work_order_report"]

def clean_colname(dataframe):
    #force column names to be lower case, no spaces, no dashes
    dataframe.columns = [x.lower().replace(" ", "_") for x in dataframe.columns]
    
    #processing data
    replacements = {
       'timedelta64[ns]': 'varchar',
        'object': 'varchar',
        'float64': 'float',
        'int64': 'integer',
        'datetime64': 'timestamp',
        'int32': 'integer'
    }

    col_str = ", ".join("{} {}".format(n, d) for (n, d) in zip(dataframe.columns, dataframe.dtypes.replace(replacements)))
    
    return col_str, dataframe.columns

In [7]:
def upload_to_db(host, dbname, user, password, col_str, table_name, dataframe, cur):
    conn_string = "host={} dbname={} user={} password={}".format(host, dbname, user, password)
    conn = psycopg2.connect(conn_string)
    print('Opened database successfully')
    
    # Drop table with same name
    cur.execute("DROP TABLE IF EXISTS %s CASCADE;" % (table_name))

    # Create table
    cur.execute("CREATE TABLE %s (%s);" % (table_name, col_str))
    print('{0} was created successfully'.format(table_name)) 
    
    # Replace NaT values with NULL
    dataframe = dataframe.where(pd.notnull(dataframe), None)

    # Insert data into table
    for index, row in dataframe.iterrows():
        values = ', '.join(["NULL" if val is None else "'{}'".format(str(val).replace("'", "''")) for val in row.tolist()])
        cur.execute("INSERT INTO %s (%s) VALUES (%s);" % (table_name, ", ".join(dataframe.columns), values))
    print('Data was inserted successfully')
    
    cur.execute("GRANT SELECT ON TABLE %s TO public;" % table_name)
    
    conn.commit()
    print('Table {0} imported to db completed'.format(table_name))
    print()
    
    return 'Upload to database completed successfully'

# Create a cursor object
conn_string = "host=localhost dbname=AE_Work_Orders_NM user=postgres password=postgres"
conn = psycopg2.connect(conn_string)
cur = conn.cursor()

# Upload each dataframe into separate tables
for i, df in enumerate(dataframes):
    col_str, dataframe_columns = clean_colname(df)
    table_name = table_names[i]
    upload_to_db("localhost", "AE_Work_Orders_NM", "postgres", "postgres", col_str, table_name, df, cur)


# Add primary key to Table 8
cur.execute("ALTER TABLE work_order_report ADD PRIMARY KEY (work_order_id)")

# Add foreign key to Table 8
cur.execute("ALTER TABLE work_order_report ALTER COLUMN date_open TYPE date USING NULLIF(date_open, '')::date")
cur.execute("ALTER TABLE work_order_report ALTER COLUMN date_closed TYPE date USING NULLIF(date_closed, '')::date")
cur.execute("ALTER TABLE work_order_report ADD CONSTRAINT fk_work_category FOREIGN KEY (work_category) REFERENCES work_category(work_category)")
cur.execute("ALTER TABLE work_order_report ADD CONSTRAINT fk_site_id FOREIGN KEY (site_id) REFERENCES school_information(site_id)")

conn.commit()
cur.close()


Opened database successfully
work_order_report was created successfully
Data was inserted successfully
Table work_order_report imported to db completed

